# Test MCP Server (Local Mode)

This notebook demonstrates how to use the `mcp_server.py` tools directly within a LangChain workflow. 
This simulates how an Agent would interact with your Local or Cloud MCP Server.

In [ ]:
import sys
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.tools import StructuredTool
from langchain_google_vertexai import ChatVertexAI
from langgraph.prebuilt import create_react_agent

# 1. Setup Path to import mcp_server from parent directory
sys.path.append("..")

# 2. Setup Environment
load_dotenv()

# POINT DATABASE TO A LOCAL FOLDER FOR THIS NOTEBOOK
# This ensures we don't mess with deployment default paths, or we explicitly stick to 'temp_data'
# mcp_server defaults to using 'DATA_DIR' env var if set
os.environ["DATA_DIR"] = os.path.abspath("../temp_data_notebook_test")
print(f"Using Data Dir: {os.environ['DATA_DIR']}")

# 3. Import Tools directly from Server Code
# (In a real remote scenario, you would use MCPClient instead)
from mcp_server import ingest_transactions, get_database_schema, query_database, semantic_search, clear_database

# 4. Wrap as LangChain Tools
tools = [
     StructuredTool.from_function(ingest_transactions),
     StructuredTool.from_function(clear_database),
     StructuredTool.from_function(get_database_schema),
     StructuredTool.from_function(query_database),
     StructuredTool.from_function(semantic_search),
]

# 5. Initialize Agent
llm = ChatVertexAI(model="gemini-1.5-flash-001", temperature=0)
agent_executor = create_react_agent(llm, tools)

print("Agent and Tools loaded successfully!")

In [ ]:
# --- STEP 1: LOAD & INGEST DATA ---
# We will read one of the CSVs from demo_data and send it to the 'ingest_transactions' tool.

# Read Local File
csv_path = "../demo_data/Chase5282_Activity20240110_20260110_20260111.CSV"

if os.path.exists(csv_path):
    with open(csv_path, 'r') as f:
        csv_content = f.read()

    print(f"Read {len(csv_content)} bytes from CSV.")
    
    # Run the Agent to Ingest
    print("\n--- Ingesting Data via Agent ---")
    response = agent_executor.invoke(
        {"messages": [("user", f"Please clear the database first, then ingest this transaction data:\n\n{csv_content}")]}
    )
    print(response["messages"][-1].content)
else:
    print(f"File not found: {csv_path}")

In [ ]:
# --- STEP 2: TEST RETRIEVAL (RAG) ---

queries = [
    "What is the database schema?",
    "How much did I spend at McDonalds?",
    "Show me the top 3 most expensive transactions.",
    "What is my total spending on Food & Drink?" 
]

for q in queries:
    print(f"\nUser: {q}")
    result = agent_executor.invoke({"messages": [("user", q)]})
    print(f"AI: {result['messages'][-1].content}")

In [3]:
%reload_ext autoreload
%autoreload 2